In [ ]:
import cv2
import json
import numpy as np
import pandas as pd
import pickle
from aquabyte.data_access_utils import DataAccessUtils

In [ ]:
data_access_utils = DataAccessUtils('/root/data/')

<h1> Get images ready for annotation </h1>

In [ ]:
# download raw images

image_s3_bucket = 'aquabyte-axiom'
left_image_key = 'fishId=190620-4e4e0640-d4eb-405d-8fcf-57fda11d7660/date=2019-06-20/hour=12/at=2019-06-20T12:41:45.636408000Z/left_frame.jpg'
right_image_key = 'fishId=190620-4e4e0640-d4eb-405d-8fcf-57fda11d7660/date=2019-06-20/hour=12/at=2019-06-20T12:41:45.636408000Z/right_frame.jpg'

left_image_f = data_access_utils.download_from_s3(image_s3_bucket, left_image_key)
right_image_f = data_access_utils.download_from_s3(image_s3_bucket, right_image_key)

# download stereo parameters

stereo_parameters_s3_bucket = 'aquabyte-stereo-parameters'
stereo_parameters_key = 'L40020185_R40020187/2019-07-02T00:00:00Z_L40020185_R40020187_stereo-parameters.json'
# stereo_parameters_key = 'L40020185_R40020187/2019-05-20T00:00:00Z_L40020185_R40020187_stereo-parameters.json'
stereo_parameters_f = data_access_utils.download_from_s3(stereo_parameters_s3_bucket, stereo_parameters_key)


In [ ]:
# define rectification utils (should be imported from Aquabyte library)

def load_params(stereo_params_file):
    """ load rectification parameters and create maps"""
    params = json.load(open(stereo_params_file))
    cameraMatrix1 = np.array(params['CameraParameters1']['IntrinsicMatrix']).transpose()
    cameraMatrix2 = np.array(params['CameraParameters2']['IntrinsicMatrix']).transpose()

    distCoeffs1 = params['CameraParameters1']['RadialDistortion'][0:2] + \
                  params['CameraParameters1']['TangentialDistortion'] + \
                  [params['CameraParameters1']['RadialDistortion'][2]]
    distCoeffs1 = np.array(distCoeffs1)

    distCoeffs2 = params['CameraParameters2']['RadialDistortion'][0:2] + \
                  params['CameraParameters2']['TangentialDistortion'] + \
                  [params['CameraParameters2']['RadialDistortion'][2]]
    distCoeffs2 = np.array(distCoeffs2)

    R = np.array(params['RotationOfCamera2']).transpose()
    T = np.array(params['TranslationOfCamera2']).transpose()
    imageSize = (4096, 3000)
    # rectification
    (R1, R2, P1, P2, Q, leftROI, rightROI) = cv2.stereoRectify(cameraMatrix1, distCoeffs1, cameraMatrix2,
                                                               distCoeffs2, imageSize, R, T, None, None, None, None,
                                                               None, cv2.CALIB_ZERO_DISPARITY, 0)

    left_maps = cv2.initUndistortRectifyMap(cameraMatrix1, distCoeffs1, R1, P1, imageSize, cv2.CV_16SC2)
    right_maps = cv2.initUndistortRectifyMap(cameraMatrix2, distCoeffs2, R2, P2, imageSize, cv2.CV_16SC2)

    return left_maps, right_maps

def stereo_rectify(stereo_parameters_f, left_frame_f, right_frame_f, 
                   rectified_left_frame_f, rectified_right_frame_f):
    left_maps, right_maps = load_params(stereo_parameters_f)    

    left_frame = cv2.imread(left_frame_f)
    rectified_left_frame = cv2.remap(left_frame, left_maps[0], left_maps[1], cv2.INTER_LANCZOS4)
    cv2.imwrite(rectified_left_frame_f, rectified_left_frame)

    right_frame = cv2.imread(right_frame_f)
    rectified_right_frame = cv2.remap(right_frame, right_maps[0], right_maps[1], cv2.INTER_LANCZOS4)
    cv2.imwrite(rectified_right_frame_f, rectified_right_frame)


In [ ]:
# rectify the images

rectified_left_image_f = left_image_f.replace('left_frame.jpg', 'rectified_left_frame.jpg')
rectified_right_image_f = right_image_f.replace('right_frame.jpg', 'rectified_right_frame.jpg')
stereo_rectify(stereo_parameters_f, left_image_f, right_image_f, rectified_left_image_f, rectified_right_image_f)

In [ ]:
stereo_parameters_f

<h1> Load in the Labelbox annotations </h1>

In [ ]:
body_parts = [
    'UPPER_LIP',
    'TAIL_NOTCH',
    'DORSAL_FIN',
    'PELVIC_FIN',
    'PECTORAL_FIN',
    'ADIPOSE_FIN',
    'ANAL_FIN',
    'EYE'
]

annotations_f = '/root/data/underwater_live_gtsf_axiom_calibration.json'
annotations = json.load(open(annotations_f))

keypoints_df = pd.DataFrame()
for idx, obj in enumerate(annotations):
    if obj['Label'] == 'Skip':
        continue

    if not all([key in obj['Label'].keys() for key in body_parts]):
        continue

    # get image file name and epoch
    camera = 'left' if 'left' in obj['Labeled Data'] else 'right'

    for body_part in body_parts:
        kp_dict = obj['Label'][body_part][0]['geometry']
        kp = (kp_dict['x'], kp_dict['y'])

        row = {
            'body_part': body_part,
            'camera': camera,
            'keypoint': kp
        }

        keypoints_df = keypoints_df.append(row, ignore_index=True)


In [ ]:
def convert_to_world_point(x, y, d, pixel_count_width, 
                           pixel_count_height, image_sensor_width, 
                           image_sensor_height, focal_length):
    """ from pixel coordinates to world coordinates """
    
    image_center_x = pixel_count_width / 2.0  
    image_center_y = pixel_count_height / 2.0
    px_x = x - image_center_x
    px_z = image_center_y - y

    sensor_x = px_x * (image_sensor_width / pixel_count_width)
    sensor_z = px_z * (image_sensor_height / pixel_count_height)

    world_y = d
    world_x = (world_y * sensor_x) / focal_length
    world_z = (world_y * sensor_z) / focal_length
    return [world_x, world_y, world_z]



def depth_from_disp(disp, focal_length_pixel, baseline):
    """ calculate the depth of the point based on the disparity value """
    depth = focal_length_pixel*baseline / np.array(disp)
    return depth


def euclidean_distance(p1, p2):
    return ((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2 + (p1[2] - p2[2])**2)**0.5

In [ ]:
stereo_frame_pairs_df = pd.DataFrame()
stereo_params = json.load(open(stereo_parameters_f))
focal_length_pixel = stereo_params['CameraParameters1']['FocalLength'][0]
baseline = abs(stereo_params['TranslationOfCamera2'][0] / 1e3) # convert millimeters to meters and use absolute value
pixel_size_m = 3.45 * 1e-6
focal_length = focal_length_pixel * pixel_size_m
image_sensor_width = 0.01412
image_sensor_height = 0.01035
pixel_count_width = 4096
pixel_count_height = 3000

epoch_mask = keypoints_df.body_part == body_part
row = {}
row['body_part'] = body_part

left_keypoints, right_keypoints, world_keypoints = {}, {}, {}

for body_part in body_parts:
    left_row = keypoints_df[(keypoints_df.body_part == body_part) & (keypoints_df.camera == 'left')].iloc[0]
    lkp = left_row['keypoint']
    left_keypoints[body_part] = lkp

    right_row = keypoints_df[(keypoints_df.body_part == body_part) & (keypoints_df.camera == 'right')].iloc[0]
    rkp = right_row['keypoint']
    right_keypoints[body_part] = rkp

    d = abs(lkp[0] - rkp[0])

    # compute world key point
    depth = depth_from_disp(d, focal_length_pixel, baseline)
    wkp = convert_to_world_point(lkp[0], lkp[1], depth, pixel_count_width, 
                                 pixel_count_height, image_sensor_width, 
                                 image_sensor_height, focal_length)

    world_keypoints[body_part] = wkp

row['left_keypoints'] = left_keypoints
row['right_keypoints'] = right_keypoints
row['world_keypoints'] = world_keypoints

stereo_frame_pairs_df = stereo_frame_pairs_df.append(row, ignore_index=True)

In [ ]:
def coord2biomass_linear(world_keypoints, model):
    """from coordinates to biomass"""

    mean = model['mean']
    std= model['std']
    PCA_components = model['PCA_components']
    reg_coef = model['reg_coef']
    reg_intercept = model['reg_intercept']
    body_parts = model['body_parts']
    print(body_parts)
    # calculate pairwise distances for production coord
    # based on the exact ordering reflected in the body_parts
    # variable above

    pairwise_distances = []
    for i in range(len(body_parts)-1):
        for j in range(i+1, len(body_parts)):
            dist = euclidean_distance(world_keypoints[body_parts[i]], world_keypoints[body_parts[j]])
            pairwise_distances.append(dist)

    interaction_values_quadratic = []
    for i in range(len(pairwise_distances)):
        for j in range(i, len(pairwise_distances)):
            dist1 = pairwise_distances[i]
            dist2 = pairwise_distances[j]
            interaction_values_quadratic.append(dist1 * dist2)

    interaction_values_cubic = []
    for i in range(len(pairwise_distances)):
        for j in range(i, len(pairwise_distances)):
            for k in range(j, len(pairwise_distances)):
                dist1 = pairwise_distances[i]
                dist2 = pairwise_distances[j]
                dist3 = pairwise_distances[k]
                interaction_values_cubic.append(dist1 * dist2 * dist3)


    X = np.array(pairwise_distances + interaction_values_quadratic + interaction_values_cubic)

    X_normalized = (X - model['mean']) / model['std']
    X_transformed = np.dot(X_normalized, model['PCA_components'].T)
    prediction = np.dot(X_transformed, reg_coef) + reg_intercept
    return prediction

In [ ]:
model = pickle.load(open('/root/data/alok/biomass_estimation/models/20190715_model_4_eig.pkl', 'rb'))
world_keypoints = stereo_frame_pairs_df.world_keypoints.iloc[0]
biomass = coord2biomass_linear(world_keypoints, model)

In [ ]:
biomass